In [161]:
import os
os.chdir('/home/nmaruani/ShapeShifter')


In [162]:
from src.diffusion.train_diffusion_colors import *
import yaml
from IPython.display import clear_output


In [163]:
with open('/home/nmaruani/ShapeShifter/configs/train_diffusion_0.yaml', 'r') as f:
    cfg = yaml.load(f, Loader=yaml.Loader)

X0 = get_gt_data(cfg, 0, "fighting-pillar")
device='cuda'

In [ ]:
L = []
LOSS_EMA = None
model = DiffusionCNN(channels=cfg["features"], layers=cfg["layers"], time_emb=cfg["time_emb"],
                        one_layers=cfg["one_layers"], first_ks=cfg["first_ks"],
                        in_channels=X0.jdata.shape[-1], out_channels=X0.jdata.shape[-1]).to(device)
mt.count_parameters(model)

optimizer = torch.optim.Adam(model.parameters(),
                                lr=cfg["lr"]
                                )

loss_fn = torch.nn.functional.mse_loss

In [165]:
def train_epoch(optimizer, model):
    global LOSS_EMA
    
    x_1 = X0.jdata
    x_0 = torch.randn_like(X0.jdata)
    t = torch.rand(X0.grid_count, 1, device=X0.device)
    t = t[X0.feature.jidx.long()]
    x_t = (1 - t) * x_0 + t * x_1    
    XT = fvnn.VDBTensor(grid=X0.grid, feature=X0.grid.jagged_like(x_t))
    optimizer.zero_grad()
    loss = loss_fn(model(XT, t.flatten()).jdata, x_1) # REAL FM is (x1-x0)
    loss.backward()
    optimizer.step()
    if LOSS_EMA is None:
        LOSS_EMA = loss.item()
    else:
        LOSS_EMA = 0.99 * LOSS_EMA + 0.01 * loss.item()
    L.append(LOSS_EMA)
train_epoch(optimizer, model)

In [ ]:
model.train()
for i in tqdm(range(cfg["epochs"])):
    train_epoch(optimizer, model)
    if i % cfg["save_every"] == 0 or i == cfg["epochs"]-1:
        clear_output(True)
        plt.plot(L, label='canyon')
        plt.yscale('log')
        plt.legend()
        plt.show()

In [167]:
### Sampling+

In [ ]:
# def model_step(model, XT, t_start, t_end):
#     with torch.no_grad():
#         TSTART = t_start.view(1).expand(len(XT.jdata))        
#         F1 = model(XT, TSTART)
#         F1.feature.jdata *= (t_end - t_start) / 2
#         F1.feature.jdata += XT.jdata
        
#         F2 = model(F1, TSTART + (t_end - t_start) / 2)
#         F2.feature.jdata *= (t_end - t_start)
#         F2.feature.jdata += XT.jdata
        
#         return F2
    
def model_step(model, XT, X0data, t_start, t_end):
    with torch.no_grad():
        TSTART = t_start.view(1).expand(len(XT.jdata))        
        F1 = model(XT, TSTART)
        F1.feature.jdata -= X0data
        F1.feature.jdata *= (t_end - t_start) / 2
        F1.feature.jdata += XT.jdata
        
        F2 = model(F1, TSTART + (t_end - t_start) / 2)
        F2.feature.jdata -= X0data
        F2.feature.jdata *= (t_end - t_start)
        F2.feature.jdata += XT.jdata
        
        return F2
        
        
i=0
n_steps = 200
time_steps = torch.linspace(0, 1.0, n_steps + 1, device=device)

X = fvnn.VDBTensor(grid=X0.grid, feature=X0.grid.jagged_like(torch.randn_like(X0.jdata)))
X0data = X.jdata.clone()
for i in tqdm(range(n_steps)):
    X = model_step(model, X, X0data, time_steps[i], time_steps[i+1])

In [ ]:
for ind in range(4):
    disp_d = DiffusionTensor(X.grid[ind], X.feature[ind]).remove_mask()
    # disp_d.get_global().colored_PC(.07)

    c=(disp_d.feature.jdata[:, 6:9].cpu().detach().numpy()+2)/4
    plot(*grid_to_mesh(disp_d.grid, colors=c), shading={'wireframe':True})